[![Open In Binder](https://static.mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/OleBo/MathSo/main?filepath=/notebooks/loesungen12.ipynb)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/OleBo/MathSo/blob/main/notebooks/loesungen12.ipynb)


[browse](http://colab.research.google.com/github/OleBo/MathSo/)

# Lösungen 12
## Aufgabe 2
### a)

In [1]:
from math import *
import numpy as np

$A = D - L - U$ mit

In [2]:
D = np.diag(np.array([5., 6., 4., 7.]))
L = np.array([[0, 0, 0, 0], [-2., 0, 0, 0], [1., -2., 0, 0], [2., 3., -2., 0]])
U = np.array([[0, 3., 0, -2.], [0, 0, 3., 0], [0, 0, 0, 1.], [0, 0, 0, 0]])

In [3]:
#Jacobi
TJ = np.linalg.inv(D).dot(L + U)
print("T_J = \n {}".format(TJ))
lamJ, vJ = np.linalg.eig(TJ)
sigmaJ = max(abs(lamJ))
print("sigma(T_J) = {}".format(sigmaJ))

T_J = 
 [[ 0.          0.6         0.         -0.4       ]
 [-0.33333333  0.          0.5         0.        ]
 [ 0.25       -0.5         0.          0.25      ]
 [ 0.28571429  0.42857143 -0.28571429  0.        ]]
sigma(T_J) = 0.8391079803591782


In [4]:
#Gauss-Seidel
TGS = np.linalg.inv(D - L).dot(U)
print("T_GS = \n {}".format(TGS))
lamGS, vGS = np.linalg.eig(TGS)
sigmaGS = max(abs(lamGS))
print("sigma(T_GS) = {}".format(sigmaGS))

T_GS = 
 [[ 0.          0.6         0.         -0.4       ]
 [ 0.         -0.2         0.5         0.13333333]
 [ 0.          0.25       -0.25        0.08333333]
 [ 0.          0.01428571  0.28571429 -0.08095238]]
sigma(T_GS) = 0.5780330470002053


$\sigma(T_J), \sigma(T_{GS}) < 1$, daher konvergieren beide Verfahren.

### b)
Nach der Bemerkung zu Satz 8.5 soll nach $K$ Iterationen gelten:
$$ \sigma(T)^K < 0.1 \Leftrightarrow K > -\frac{1}{\log_{10} \sigma(T)}. $$ 

In [5]:
#Jacobi: 
KJ = -1./log10(sigmaJ)
print("K_J = {}".format(ceil(KJ)))

K_J = 14


In [6]:
#Gauss-Seidel:
KGS = -1./log10(sigmaGS)
print("K_GS = {}".format(ceil(KGS)))

K_GS = 5


### c)
Aus notebook12.ipynb übernommen:

In [7]:
def jor(A, b, omega, x0, tol, maxits=1000, verbose=False, exact=None):
    """Löst das LGS Ax=b mithilfe der JOR-Iteration mit Startvektor x0 und Relaxationsparameter omega. 
    Bricht ab, wenn die 2-Norm des Residuums kleiner als tol ist. Optional kann die exakte Lösung übergeben
    werden, dann wird bei verbose=True auch die 2-Norm des Fehlers ausgegeben.
    Vektoren werden hier als Zeilenvektoren behandelt."""
    n = len(x0)
    x = x0.copy()
    res = np.infty
    k = 1
    while k <= maxits and res > tol:
        xnew = (1. - omega)*x
        for i in range(n):
            xnew[i] += omega*b[i]/A[i,i]
            noti = [j for j in range(i)] + [j for j in range(i+1, n)]
            for j in noti:
                xnew[i] -= omega*A[i,j]*x[j]/A[i,i]
        x = xnew
        res = np.linalg.norm(x.dot(A.T) - b)  #beachte x, b sind Zeilenvektoren, (A x^T)^T = x A^T 
        if verbose:
            if exact is not None:
                err = np.linalg.norm(x - exact)
            else:
                err = 0.
            print("k = {:3d}, res = {:.2e}, err = {:.2e}".format(k, res, err))
        k += 1
    return x

In [8]:
def sor(A, b, omega, x0, tol, maxits=1000, verbose=False, exact=None):
    """Löst das LGS Ax=b mithilfe der SOR-Iteration mit Startvektor x0 und Relaxationsparameter omega. 
    Bricht ab, wenn die 2-Norm des Residuums kleiner als tol ist. Optional kann die exakte Lösung übergeben
    werden, dann wird bei verbose=True auch die 2-Norm des Fehlers ausgegeben.
    Vektoren werden hier als Zeilenvektoren behandelt."""
    n = len(x0)
    x = x0.copy()
    res = np.infty
    k = 1
    while k <= maxits and res > tol:
        for i in range(n):
            xnewi = (1. - omega)*x[i] + omega*b[i]/A[i,i]
            noti = [j for j in range(i)] + [j for j in range(i+1, n)]
            for j in noti:
                xnewi -= omega*A[i,j]*x[j]/A[i,i]  #verwendet für j<i bereits die aktualisierten x[j]
            x[i] = xnewi
        res = np.linalg.norm(x.dot(A.T) - b)  #beachte x, b sind Zeilenvektoren, (A x^T)^T = x A^T 
        if verbose:
            if exact is not None:
                err = np.linalg.norm(x - exact)
            else:
                err = 0.
            print("k = {:3d}, res = {:.2e}, err = {:.2e}".format(k, res, err))
        k += 1
    return x

In [9]:
A = np.array([[5., -3., 0., 2.], [2., 6., -3., 0.], [-1, 2., 4., -1.], [-2., -3., 2., 7.]])
b = np.array([13., 16., -11., 10.])
x_ex = np.array([2., 1., -2., 3.])
x0 = np.array([1., 1., 1., 1.])
tol = 1e-6
#Jacobi-Verfahren
x = jor(A, b, 1., x0, tol, verbose=True, exact=x_ex)

k =   1, res = 2.26e+01, err = 2.42e+00
k =   2, res = 1.57e+01, err = 2.31e+00
k =   3, res = 1.50e+01, err = 1.75e+00
k =   4, res = 1.21e+01, err = 1.55e+00
k =   5, res = 9.65e+00, err = 1.30e+00
k =   6, res = 9.19e+00, err = 1.03e+00
k =   7, res = 6.28e+00, err = 9.55e-01
k =   8, res = 6.67e+00, err = 7.01e-01
k =   9, res = 4.40e+00, err = 6.80e-01
k =  10, res = 4.58e+00, err = 4.99e-01
k =  11, res = 3.34e+00, err = 4.67e-01
k =  12, res = 2.99e+00, err = 3.68e-01
k =  13, res = 2.58e+00, err = 3.12e-01
k =  14, res = 1.92e+00, err = 2.74e-01
k =  15, res = 1.93e+00, err = 2.09e-01
k =  16, res = 1.28e+00, err = 1.99e-01
k =  17, res = 1.37e+00, err = 1.44e-01
k =  18, res = 9.29e-01, err = 1.39e-01
k =  19, res = 9.21e-01, err = 1.05e-01
k =  20, res = 7.15e-01, err = 9.45e-02
k =  21, res = 5.94e-01, err = 7.77e-02
k =  22, res = 5.48e-01, err = 6.29e-02
k =  23, res = 3.84e-01, err = 5.74e-02
k =  24, res = 4.02e-01, err = 4.25e-02
k =  25, res = 2.64e-01, err = 4.11e-02


In [10]:
#Gauss-Seidel-Verfahren
x = sor(A, b, 1., x0, tol, verbose=True, exact=x_ex)

k =   1, res = 1.24e+01, err = 2.01e+00
k =   2, res = 5.52e+00, err = 9.64e-01
k =   3, res = 3.41e+00, err = 5.78e-01
k =   4, res = 1.98e+00, err = 3.40e-01
k =   5, res = 1.15e+00, err = 1.97e-01
k =   6, res = 6.65e-01, err = 1.14e-01
k =   7, res = 3.85e-01, err = 6.61e-02
k =   8, res = 2.22e-01, err = 3.82e-02
k =   9, res = 1.29e-01, err = 2.21e-02
k =  10, res = 7.43e-02, err = 1.28e-02
k =  11, res = 4.30e-02, err = 7.38e-03
k =  12, res = 2.48e-02, err = 4.26e-03
k =  13, res = 1.44e-02, err = 2.46e-03
k =  14, res = 8.30e-03, err = 1.42e-03
k =  15, res = 4.80e-03, err = 8.23e-04
k =  16, res = 2.77e-03, err = 4.76e-04
k =  17, res = 1.60e-03, err = 2.75e-04
k =  18, res = 9.26e-04, err = 1.59e-04
k =  19, res = 5.35e-04, err = 9.19e-05
k =  20, res = 3.10e-04, err = 5.31e-05
k =  21, res = 1.79e-04, err = 3.07e-05
k =  22, res = 1.03e-04, err = 1.78e-05
k =  23, res = 5.98e-05, err = 1.03e-05
k =  24, res = 3.46e-05, err = 5.93e-06
k =  25, res = 2.00e-05, err = 3.43e-06


Die Zahl der Iterationen, die für die Abnahme des Fehlers um eine Zehnerpotenz benötigt werden, passen sehr gut zu den Vorhersagen in b).